<a href="https://colab.research.google.com/github/MoazAbbas/DES-in-CBC-mode-Encryption/blob/main/MediaProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageDraw
from scipy.ndimage import convolve
# Load the CSV as a numpy array
pixels = np.loadtxt("/content/secret_image .csv", delimiter=',')
pixels = pixels.astype(np.uint8)

# Create and save grayscale image
img = Image.fromarray(pixels, mode='L')
img.save("output_image.png")

# Save color-mapped versions
plt.imsave('image_plasma.png', pixels, cmap='plasma')
plt.imsave('image_hot.png', pixels, cmap='hot')


In [92]:
#phase 2
black_pixel_coords = np.argwhere(pixels == 0)

# Count black pixels
black_pixel_count = len(black_pixel_coords)
print(f"Number of black pixels: {black_pixel_count}")

# Print coordinates
print("Coordinates of black pixels:")
for y, x in black_pixel_coords:  # note: rows = y, cols = x
    print(f"({x}, {y})")

# Bounding box (min/max x and y)
if black_pixel_count > 0:
    ys, xs = black_pixel_coords[:, 0], black_pixel_coords[:, 1]
    min_x, max_x = xs.min(), xs.max()
    min_y, max_y = ys.min(), ys.max()
    print(f"Bounding box:")
    print(f"  min_x: {min_x}, max_x: {max_x}")
    print(f"  min_y: {min_y}, max_y: {max_y}")
else:
    print("No black pixels found.")

Number of black pixels: 9
Coordinates of black pixels:
(5, 5)
(14, 5)
(6, 13)
(12, 13)
(7, 14)
(8, 14)
(9, 14)
(10, 14)
(11, 14)
Bounding box:
  min_x: 5, max_x: 14
  min_y: 5, max_y: 14


In [93]:

def get_shape_pixels(image_array):
    rows, cols = image_array.shape
    shape_pixels = []

    # Find connected black pixels with black neighbors
    for r in range(rows):
        for c in range(cols):
            if image_array[r, c] == 0:  # black pixel
                neighbors = [
                    (r - 1, c), (r + 1, c),
                    (r, c - 1), (r, c + 1),
                    (r - 1, c - 1), (r - 1, c + 1),
                    (r + 1, c - 1), (r + 1, c + 1)
                ]
                for nr, nc in neighbors:
                    if 0 <= nr < rows and 0 <= nc < cols and image_array[nr, nc] == 0:
                        shape_pixels.append((r, c))
                        break
    return shape_pixels

def rotate_pixels_180(pixels, center):
    return [(2*center[0] - r, 2*center[1] - c) for r, c in pixels]

pixels = np.clip(pixels, 0, 255).astype(np.uint8)

# Copy the original image (for non-destructive transformation)
original_pixels = pixels.copy()

# Find the shape pixels (black pixels with black neighbors)
shape_pixels = get_shape_pixels(pixels)

# Make the shape white in the original image
for r, c in shape_pixels:
    original_pixels[r, c] = 255  # Set the shape pixels to white

# Calculate center of the shape
r_coords, c_coords = zip(*shape_pixels)
center_r = sum(r_coords) / len(r_coords)
center_c = sum(c_coords) / len(c_coords)
center = (center_r, center_c)

# Rotate the shape 180 degrees
rotated_shape = rotate_pixels_180(shape_pixels, center)

# Create a new copy for the rotated shape (to keep the original intact)
rotated_image = original_pixels.copy()

# Place the rotated shape back at the same location in the rotated image
for r, c in rotated_shape:
    if 0 <= int(r) < rotated_image.shape[0] and 0 <= int(c) < rotated_image.shape[1]:
        rotated_image[int(r), int(c)] = 0  # Set rotated pixels back to black
# Show and save result
final_img = Image.fromarray(rotated_image)
#--------------------------------------------------------------------------------------------------------------
def find_isolated_black_pixels(image_array):
    rows, cols = image_array.shape
    masked_eye = []

    for r in range(rows):
        for c in range(cols):
            if image_array[r, c] == 0:
                is_isolated = True
                neighbors = [
                    (r - 1, c), (r + 1, c),
                    (r, c - 1), (r, c + 1),
                    (r - 1, c - 1), (r - 1, c + 1),
                    (r + 1, c - 1), (r + 1, c + 1)
                ]
                for nr, nc in neighbors:
                    if 0 <= nr < rows and 0 <= nc < cols and image_array[nr, nc] == 0:
                        is_isolated = False
                        break
                if is_isolated:
                    masked_eye.append((r, c))
    return masked_eye

rotated_image_rgb = np.stack([rotated_image] * 3, axis=-1)  # Convert to 3D array (height, width, 3)

# Find isolated black pixels
masked_eye = find_isolated_black_pixels(rotated_image)

# Mark red isolated pixels
for r, c in masked_eye:
    rotated_image_rgb[r, c] = [255, 0, 0]  # Red (RGB)

# Add 1-pixel-wide blue border
rows, cols = rotated_image_rgb.shape[:2]
for r in range(rows):
    for c in range(cols):
        if r == 0 or r == rows - 1 or c == 0 or c == cols - 1:
            rotated_image_rgb[r, c] = [0, 0, 255]  # Blue
#------------------------------------------------------------------------------------------------------------------
final_img = Image.fromarray(rotated_image_rgb)
final_img.show()

# Save the final image
final_img.save("rotated_pixels_with_red_and_blue.png")

In [94]:
#phase 4
img = Image.open("/content/rotated_pixels_with_red_and_blue.png")
img_array = np.array(img)

# Define a 3x3 mean filter kernel
mean_kernel = np.ones((3, 3)) / 9

# Apply the filter to each RGB channel
filtered_array = np.zeros_like(img_array)
for i in range(3):  # for R, G, B
    filtered_array[:, :, i] = convolve(img_array[:, :, i], mean_kernel, mode='nearest')

# Convert back to image and display
filtered_img = Image.fromarray(filtered_array.astype(np.uint8))
filtered_img.show()
filtered_img.save("final_mean_filtered.png")

[link text](https://)

In [95]:
#phase 5
print("How many black pixels were found?")
print(f"Number of black pixels: {black_pixel_count}")
print("● What are the coordinates of the black pixels?")
print("Coordinates of black pixels:")
for y, x in black_pixel_coords:  # note: rows = y, cols = x
    print(f"({x}, {y})")
print("● What is the bounding box?")
print(f"Bounding box:")
print(f"  min_x: {min_x}, max_x: {max_x}")
print(f"  min_y: {min_y}, max_y: {max_y}")

print("● What features did you detect in the image?")


How many black pixels were found?
Number of black pixels: 9
● What are the coordinates of the black pixels?
Coordinates of black pixels:
(5, 5)
(14, 5)
(6, 13)
(12, 13)
(7, 14)
(8, 14)
(9, 14)
(10, 14)
(11, 14)
● What is the bounding box?
Bounding box:
  min_x: 5, max_x: 14
  min_y: 5, max_y: 14
● What features did you detect in the image?
